======================== Import Packages ==========================

In [1]:
import sys, pdb, glob
import numpy as np
from astropy.table import Table, join
from astroquery.vizier import Vizier
import warnings
from astropy.logger import AstropyWarning
warnings.filterwarnings('ignore', category=AstropyWarning)
from IPython.display import display, Latex

===================== Define Functions ===================

In [2]:
def get_data(catalog, join_key='Name'):

    """
    PURPOSE:    Get data from literature with Vizier

    INPUT:      catalog = ctalog name on Vizier (str)
                join_key = column header to join tables, if multiple (str; optional)

    OUTPUT:     t = data table (AstroPy Table)

    """

    ### GET FULL CATALOG (ALL COLUMNS, ALL ROWS)
    viz = Vizier(catalog=catalog, columns=['**'])
    viz.ROW_LIMIT = -1
    tv = viz.get_catalogs(catalog)

    ### IF MULTIPLE TABLES, JOIN THEN
    for i, val in enumerate(tv.keys()):
        if i == 0:
            t = tv[val]
        else:
            tt = tv[val]
            if join_key in tt.columns:
                t = join(t, tt, join_type='inner', keys=join_key)

    return t

========================== Code ==========================

In [3]:
### LOAD IN LUPUS DATA
T = get_data("J/ApJ/828/46")
T.sort('RAJ2000')

In [4]:
### WRITE HEADER INFO
f = open('../output/table_01.tex', 'w')
f.write(r'\capstartfalse'                                     + ' \n')
f.write(r'\begin{deluxetable}{lrrrr}'                         + ' \n')
f.write(r'\tabletypesize{\footnotesize}'                      + ' \n')
f.write(r'\centering'                                         + ' \n')
f.write(r'\tablewidth{240pt}'                                 + ' \n')
f.write(r'\tablecaption{Stellar Properties \label{tab-star}}' + ' \n')
f.write(r'\tablecolumns{5} '                                  + ' \n')
f.write(r'\tablehead{'                                        + ' \n')
f.write(r' \colhead{Source}'                                  + ' \n')
f.write(r'&\colhead{$d$ (pc)}'                                + ' \n')
f.write(r'&\colhead{SpT}'                                     + ' \n')
f.write(r'&\colhead{$M_{\ast}$/$M_{\odot}$}'                  + ' \n')
f.write(r'&\colhead{Ref}'                                     + ' \n')
f.write(r'}'                                                  + ' \n')
f.write(r'\startdata'                                         + ' \n')

12

In [5]:
### WRITE FIRST 10 LINES
lim = 10
for i, val in enumerate(np.zeros(lim)):

    source = T['Name'][i]
    dist = str(T['Dist'][i])

    if np.ma.is_masked(T['Ref'][i]) is True:
        ref = '...'
    else:
        ref = str(T['Ref'][i])

    if T['SpT'][i] == '':
        spt = '...'
    else:
        spt = T['SpT'][i]

    if np.ma.is_masked(T['Mass'][i]) is True:
        mstar = '...'
    else:
        mstar  = str(T['Mass'][i]) + r' $\pm$ ' + str(T['e_Mass'][i])

    ### END OF LINE
    if (i < lim - 1):
        end = r' \\' + '\n'
    else:
        end = '\n'

    f.write(source + ' & ' + dist + ' & ' + spt + ' & ' + mstar + ' & ' + ref + end)

In [8]:
table_header = r'\begin{array}{lrrrr} \hline' + '\n'
table_header += r'\text{Source} & d\text{ (pc)} & \text{SpT} & M_{\ast}/M_{\odot} & \text{Ref} \\' + '\n'
table_header += r'\hline' + '\n'

table_body = ''
lim = 10
for i, val in enumerate(np.zeros(lim)):
    source = T['Name'][i]
    dist = str(T['Dist'][i])

    if np.ma.is_masked(T['Ref'][i]) is True:
        ref = '...'
    else:
        ref = str(T['Ref'][i])

    if T['SpT'][i] == '':
        spt = '...'
    else:
        spt = T['SpT'][i]

    if np.ma.is_masked(T['Mass'][i]) is True:
        mstar = '...'
    else:
        mstar  = str(T['Mass'][i]) + r' \pm ' + str(T['e_Mass'][i])

    table_body += source + ' & ' + dist + ' & ' + spt + ' & ' + mstar + ' & ' + ref + r' \\' + '\n'

table_footer = r'\hline' + '\n'
table_footer += r'\end{array}' + '\n'

table = table_header + table_body + table_footer

display(Latex(table))

<IPython.core.display.Latex object>

In [7]:
### WRITE FOOTER INFO
f.write(r'\enddata' + ' \n')
f.write(r'\tablenotetext{}{References:' +
        r' (1) \cite{2014A&A...561A...2A},' +
        r' (2) Alcal\'{a} et al. (in prep),' +
        r' (3) \cite{2013MNRAS.429.1001A},' +
        r' (4) \cite{2011MNRAS.418.1194M},' +
        r' (5) \cite{2008ApJS..177..551M},' +
        r' (6) Cleeves et al. (in prep),' +
        r' (7) \cite{2015A&A...578A..23B},' +
        r' (8) \cite{2008hsf2.book..295C}.' +
        r' (This table is available in its entirety in machine-readible form.)}' + ' \n')
f.write(r'\end{deluxetable}' + ' \n')
f.write(r'\capstartfalse' + ' \n')
f.close()